In [1]:
with open('suihanki_wakati.txt', mode='r', encoding='utf-8') as f:
    text = f.read()

In [2]:
# 共起行列(と単語のリスト)を返す
# w_size: ウィンドウ幅
def cooccurrence(text, w_size):
    import numpy as np

    # i文目のj語目が格納された二次元配列sentence(i, j)
    sentence = text.split('\n')
    for i in range(len(sentence)):
        sentence[i] = sentence[i].split()

    # 登場した単語のリスト
    words = list(set(text.split()))

    # words数サイズのゼロ行列を作る
    matrix = np.zeros((len(words), len(words)))

    # number番目の単語がw
    for number, w in enumerate(words):
        for i in range(len(sentence)):
            for j in range(len(sentence[i])):
                # wとi文目j単語目とが合致したら
                if w == sentence[i][j]:
                    # 行列の、窓幅分前の単語のところを+1する
                    for d in range(1, w_size+1):
                        if j-d >= 0:
                            matrix[number][words.index(sentence[i][j-d])] += 1
                    # 行列の、窓幅分後の単語のところを+1する
                    for d in range(1, w_size+1):
                        if j+d < len(sentence[i]):
                            matrix[number][words.index(sentence[i][j+d])] += 1

    return words, matrix

In [3]:
# ベクトルの0でない成分を返す
# wは単語ベクトル
def F(w):
    a = []
    for n, i in enumerate(w):
        if i != 0:
            a.append(n)
    return set(a)

In [4]:
def WeedsP(w1, w2):
    m = 0
    for i in list(F(w1)):
        m += w1[i]
    
    if m == 0:
        return 0

    c = 0
    for i in list(F(w1)&F(w2)):
        c += w1[i]
    return c / m

In [5]:
def weeds_list(text, w_size=1):
    # words: textに登場する単語のリスト
    # matrix: 共起行列
    words, matrix = cooccurrence(text, w_size=1)
    
    # 下位語候補, 上位語候補, weeds値
    # を格納する配列を用意しておく
    result = []
    
    for i in range(len(words)):
        for j in range(len(words)):
            if i == j:
                continue
            else:
                result.append([words[i], words[j], WeedsP(matrix[i], matrix[j])])
    
    # weeds値が大きい順(上位・下位関係がはっきりしている順)に並べ替える
    result = sorted(result, reverse=True, key=lambda x: x[2])
    
    return result

In [ ]:
weeds = weeds_list(text, w_size=1)
print(weeds[:10])

In [ ]:
import numpy as np
np.savetxt('weeds.txt', np.array(weeds))